# STRMaps

You can view the map at https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html FOR NOW
File will remain at s3://requesterpays.garyscorner.net/datasets/html/  (requester pays use AWS CLI [--request-payer requester])

Datasets:
* https://data.nola.gov/Housing-Land-Use-and-Blight/Map-of-Short-Term-Rental-Licenses/j5u3-2ueh

In [1]:
import pandas as pd
import folium
import numpy as np
import folium.plugins

from io import StringIO

from json import JSONEn

from datetime import datetime

In [2]:
! curl https://data.nola.gov/api/views/en36-xvxg/rows.csv?accessType=DOWNLOAD -o DataSets\Short-Term_Rental_Permit_Applications.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  783k    0  783k    0     0   439k      0 --:--:--  0:00:01 --:--:--  439k
100 6782k    0 6782k    0     0  2454k      0 --:--:--  0:00:02 --:--:-- 2454k
100 9301k    0 9301k    0     0  2963k      0 --:--:--  0:00:03 --:--:-- 2965k


In [4]:
strDf = pd.read_csv("./DataSets/Short-Term_Rental_Permit_Applications.csv")

In [5]:
#drop without location
strDf = strDf[ strDf['X'] > 0.0]

In [6]:
strDf.drop(columns=['X','Y'],inplace=True)

In [7]:
#drop dups and withdrawn
strDf = strDf[ ~(strDf['Current Status'].isin(['Duplicate','Withdrawn'])) ]

In [8]:
strDf['Link'] = strDf['Link'].map(lambda url:  f"<a target='_blank' href='{url}'>One Stop App</a>")

In [9]:
%%time

def getPoints(val):
    try:
        
        lat,lon = val[1:-1].split(', ')
        lon = float(lon)
        lat = float(lat)
        pt = (lat,lon)
        
    except Exception as e:
        print(val)
        raise(e)
        
    return pt

strDf['Location'] = strDf['Location'].map(getPoints)


Wall time: 62.2 ms


In [10]:
strDf['Address'] = strDf.apply(lambda a: f"<a target='_blank' href='http://maps.google.com/maps?q=&layer=c&cbll={a['Location'][0]},{a['Location'][1]}'>{a['Address']}</a>",axis=1)

In [11]:
strDf['Current Status'].value_counts()

Expired    14224
Denied      7429
Pending     4006
Issued      2357
Revoked      143
Name: Current Status, dtype: int64

In [12]:
statusList = list(np.unique(strDf['Current Status']))

In [13]:
#you can uncomment that to see the map in this notebook but the notebook will balloon to 85 mb
#map

In [14]:
jsonData = StringIO()
strDf.iloc[:5].to_json(jsonData)


In [15]:
with open('STRMapTemplate.html', 'r') as f:
    templateHTML = f.read()

In [16]:
beginHTML, endHTML = templateHTML.split('DATA_ALL')

jsonData.seek(0)
with open('STRMap.html', 'w') as f:
    f.write(beginHTML)
    f.write(jsonData.read())
    f.write(endHTML)

In [16]:
print(f"Finished {datetime.now()}")

Finished 2023-07-23 22:47:11.277736
